In [0]:
%sql
create schema if not exists ecommerce.silver

In [0]:
# Importando as funções necessárias
from pyspark.sql.functions import col, sum, avg, count, min, max, first

In [0]:
# Lendo tabelas na camada silver
clientes_df = spark.read.table("ecommerce.silver.clientes")
pedidos_df = spark.read.table("ecommerce.silver.pedidos")
itens_pedidos_df = spark.read.table("ecommerce.silver.itens_pedido")
produtos_df = (
    spark.read.table("ecommerce.silver.produtos")
    .withColumnRenamed("ativo", "produto_ativo")
    .withColumnRenamed("data_criacao", "produto_data_criacao")
)
categorias_df = (
    spark.read.table("ecommerce.silver.categorias")
    .withColumnRenamed("ativo", "categoria_ativo")
    .withColumnRenamed("data_criacao", "categoria_data_criacao")
)
pagamentos_df = spark.read.table("ecommerce.silver.pagamentos")
avaliacoes_df = spark.read.table("ecommerce.silver.avaliacoes")

In [0]:
obt_df = (
    itens_pedidos_df.alias("i")
    .join(pedidos_df.alias("p"),      "pedido_id",   "left")
    .join(clientes_df.alias("c"),     "cliente_id",  "left")
    .join(produtos_df.alias("pr"),    "produto_id",  "left")
    .join(categorias_df.alias("cat"), "categoria_id","left")
    .join(pagamentos_df.alias("pg"),  "pedido_id",   "left")
    .join(avaliacoes_df.alias("av"),  "produto_id",  "left")
    .select(
        # Chaves
        col("i.item_pedido_id"),
        col("p.pedido_id"),
        col("c.cliente_id"),
        col("pr.produto_id"),

        # Pedido
        col("p.data_pedido"),
        col("p.status").alias("status_pedido"),
        col("p.valor_total"),
        col("p.valor_liquido"),
        col("p.frete"),
        col("p.desconto"),
        col("p.data_entrega_prevista"),
        col("p.data_entrega_realizada"),
        col("p.ano"),
        col("p.trimestre"),
        col("p.ticket_classificacao"),

        # Cliente
        col("c.nome").alias("nome_cliente"),
        col("c.cidade"),
        col("c.estado"),
        col("c.regiao"),
        col("c.data_cadastro"),

        # Produto
        col("pr.nome").alias("nome_produto"),
        col("pr.marca"),
        col("pr.preco"),
        col("pr.faixa_preco"),
        col("pr.produto_ativo"),

        # Categoria
        col("cat.nome").alias("nome_categoria"),
        col("cat.categoria_ativo"),

        # Item
        col("i.quantidade"),
        col("i.preco_unitario"),
        (col("i.quantidade") * col("i.preco_unitario")).alias("valor_item"),

        # Pagamento
        col("pg.pagamento_id"),
        col("pg.metodo").alias("metodo_pagamento"),
        col("pg.status").alias("status_pagamento"),
        col("pg.valor").alias("valor_pago"),
        col("pg.data_pagamento"),

        # Avaliação
        col("av.avaliacao_id"),
        col("av.nota"),
        col("av.comentario"),
        col("av.data_avaliacao")
    )
)

In [0]:
display(obt_df)

In [0]:
# Salvando a tabela na camada silver
(
    obt_df
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable("ecommerce.silver.obt_ecommerce")
)